In [1]:
%matplotlib inline
from PIL import Image
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from collections import defaultdict

from sklearn.preprocessing import MultiLabelBinarizer, MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import fbeta_score, precision_score, make_scorer, average_precision_score

import cv2

from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Input
from keras import backend as K

Using TensorFlow backend.


In [4]:
n_samples = 5000
rescaled_dim = 32

path = '/Users/theisenm'
df = pd.read_csv(path + '/Dropbox/Kaggle/train_v2.csv')
sample = pd.read_csv(path + '/Dropbox/Kaggle/sample_submission_v2.csv')
lb = MultiLabelBinarizer()


In [5]:
def set_output(df):
    df['split_tags'] = df['tags'].map(lambda row: row.split(" "))
    y = lb.fit_transform(df['split_tags'])
    y = y[:n_samples]
    return y

def set_input(train_features):
    X = np.squeeze(np.array(train_features, np.float16) / 255.)
    return X

def calculate_f2(y_test, predicted):
    score = fbeta_score(y_test, predicted, beta=2, average=None)
    avg_sample_score = fbeta_score(y_test, predicted, beta=2, average='samples')
    print('Average F2 test score {}'.format(avg_sample_score))
    #print('F2 test scores per tag:')
    #print [(lb.classes_[l], score[l]) for l in score.argsort()[::-1]]

In [6]:
# FEATURE EXTRACTION

# Raw features
#raw_features_train = [cv2.resize(cv2.imread(path + '/Kaggle/train-jpg/{}.jpg'.format(name)), (rescaled_dim, rescaled_dim)) for name in df.head(n_samples)['image_name'].values]
raw_features_train = [cv2.imread(path + '/Kaggle/train-jpg/{}.jpg'.format(name)) for name in df.head(n_samples)['image_name'].values]

#raw_features_test = [cv2.resize(plt.imread(path + '/Kaggle/test-jpg/{}.jpg'.format(name)), (rescaled_dim, rescaled_dim)).reshape(1, -1) for name in sample['image_name'].values]

In [7]:
# SET X's & Y's
y = set_output(df)
X = set_input(raw_features_train)
#X_sub = set_input(raw_features_test)

print(X.shape, y.shape, lb.classes_)
#print X_sub.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
X_train.shape

((5000, 256, 256, 3), (5000, 17), array(['agriculture', 'artisinal_mine', 'bare_ground', 'blooming',
       'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation',
       'habitation', 'haze', 'partly_cloudy', 'primary', 'road',
       'selective_logging', 'slash_burn', 'water'], dtype=object))


(3350, 256, 256, 3)

In [8]:
# create the base pre-trained model
base_model = InceptionV3(input_shape=(256,256,3), weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 17 classes
predictions = Dense(17, activation='sigmoid')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)
#print model.summary()

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')

# train the model on the new data for a few epochs
model.fit(X_train, y_train, batch_size=100, epochs=2, callbacks=[early_stopping])



print model.metrics_names
model.evaluate(X_test, y_test)

predicted = model.predict(X_test, batch_size=100, verbose=0)

/usr/local/lib/python2.7/site-packages/keras/backend/tensorflow_backend.py:2252: UserWarning:

Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend



Epoch 1/2
3350/3350 [==============================] - 592s - loss: 0.3969 - acc: 0.8837   
Epoch 2/2
3350/3350 [==============================] - 598s - loss: 0.2331 - acc: 0.9176   
['loss', 'acc']
1650/1650 [==============================] - 328s   


In [9]:
predicted = (predicted > 0.5).astype(int)
calculate_f2(y_test, predicted)

Average F2 test score 0.693567609605


/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in labels with no predicted samples.

/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

F-score is ill-defined and being set to 0.0 in samples with no predicted labels.



In [10]:
# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit(X_train, y_train, batch_size=100, epochs=2)

(0, 'input_1')
(1, 'conv2d_1')
(2, 'batch_normalization_1')
(3, 'activation_1')
(4, 'conv2d_2')
(5, 'batch_normalization_2')
(6, 'activation_2')
(7, 'conv2d_3')
(8, 'batch_normalization_3')
(9, 'activation_3')
(10, 'max_pooling2d_1')
(11, 'conv2d_4')
(12, 'batch_normalization_4')
(13, 'activation_4')
(14, 'conv2d_5')
(15, 'batch_normalization_5')
(16, 'activation_5')
(17, 'max_pooling2d_2')
(18, 'conv2d_9')
(19, 'batch_normalization_9')
(20, 'activation_9')
(21, 'conv2d_7')
(22, 'conv2d_10')
(23, 'batch_normalization_7')
(24, 'batch_normalization_10')
(25, 'activation_7')
(26, 'activation_10')
(27, 'average_pooling2d_1')
(28, 'conv2d_6')
(29, 'conv2d_8')
(30, 'conv2d_11')
(31, 'conv2d_12')
(32, 'batch_normalization_6')
(33, 'batch_normalization_8')
(34, 'batch_normalization_11')
(35, 'batch_normalization_12')
(36, 'activation_6')
(37, 'activation_8')
(38, 'activation_11')
(39, 'activation_12')
(40, 'mixed0')
(41, 'conv2d_16')
(42, 'batch_normalization_16')
(43, 'activation_16')
(44, 'c

In [11]:
raw_features_test = [cv2.imread(path + '/Kaggle/test-jpg/{}.jpg'.format(name)) for name in sample['image_name'].values]
X_sub = set_input(raw_features_test)
print X_sub.shape

(40669, 256, 256, 3)


In [13]:
# Classify test set & prepare submission
y_sub = model.predict(X_sub, batch_size=100, verbose=0)

KeyboardInterrupt: 

In [ ]:
sample = pd.read_csv(path + '/Dropbox/Kaggle/sample_submission.csv')

all_test_tags = []
for index in range(y_sub.shape[0]):
    all_test_tags.append(' '.join(list(lb.classes_[np.where(y_sub[index, :] == 1)[0]])))

sample['tags'] = all_test_tags
print sample.head()

In [ ]:
# Save output to CSV
sample.to_csv('keras_f2_{}.csv'.format(avg_sample_score), index=False)